# Modelling More Realistic Corrosion

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sympy
import scipy

$$\require{mhchem}$$

## Mass transfer limited corrosion

In the first attempt at modelling corrosion, I used a thickness loss rate per year for steel in atmospheric corrosion to calculate backwards to moles per second of iron oxidation. This is not a very realistic or physical basis for many reasons, not least of which is that atmospheric corrosion is carried out over long periods of time, where the metal is exposed to dry, humid, rain, hot, cold, etc. conditions as the weather and seasons vary. 

I would this time instead like to try to start from a more real system. I am thinking in this case iron under a layer of salt water where there corrosion rate is limited by mass transfer of oxygen. Iron corrodes readily in oxygen rich salt water, and as oxygen is reduced in the corrosion reaction an oxygen concentration gradient forms between the bulk solution and the metal surface. Assuming the oxygen is consumed faster than is can be transported to the surface, then the oxygen concentration at the surface will eventually reach zero, since it will be consumed instantly as it reaches it. The space this concentration gradient extends from the metal surface is called the Nernst layer. 

### Deriving equations for mass transfer limited corrosion

Transport of oxygen through the Nernst layer to the surface of the metal can be determined with Fick's law.

$$
J_{\ce{O2}} = -D_{\ce{O2}}\frac{dc_{\ce{O2}}}{dx}
$$

Here $J_{\ce{O2}}$ is the flux of $\ce{O2}$ to the metal surface in $\frac{mol}{cm^2s}$, $D_{\ce{O2}}$ is the diffusion coefficient of $\ce{O2}$ in $\frac{cm^2}{s}$, $c_{\ce{O2}}$ is the concentration of $\ce{O2}$ in $\frac{mol}{cm^3}$, and $x$ is the distance from the metal surface in cm. The equation can then be integrated from $x = 0$ to $x = \delta$ which gives

$$
J_{\ce{O2}} = -D_{\ce{O2}}\frac{c_{\ce{O2},b} - c_{\ce{O2},0}}{\delta}
$$

where $c_{\ce{O2},b}$ and $c_{\ce{O2},0}$ are the bulk and metal surface concentrations of $\ce{O2}$, respectively, and $\delta$ is the thickness of the Nernst layer in cm. 

Faraday's law relates the rate of the reduction reaction to current. It can be expressed in terms of total current:

$$
I_{red} = \frac{zFn}{t}
$$

or in terms of current density:

$$
i_{red} = \frac{zFn}{At}
$$

where $I_{red}$ is the reduction current in A, $i_{red}$ is the reduction current density in $\frac{A}{cm^2}$, $z$ is the number of electrons involved in the reduction reaction, $F$ is Faraday's constant, $96485 \frac{C}{mol}$, $n$ is the amount of the reduced species in mol, $t$ is time in s, and $A$ is area in $cm^2$. Since $\frac{n}{At}$ is the flux, Fick's law can be substituted into Faraday's law, so in the case of oxygen reduction:

$$
i_{\ce{red}} = \frac{zFn_{\ce{O2}}}{At} = zFJ_{\ce{O2}} = -zFD_{\ce{O2}}\frac{c_{\ce{O2},b} - c_{\ce{O2},0}}{\delta}
$$

If the reaction is limited by mass transfer, then $c_{\ce{O2},0}$ becomes 0 and the equation becomes:

$$
i_{\ce{red}} = -zFD_{\ce{O2}}\frac{c_{\ce{O2},b}}{\delta} = i_{lim}
$$

Here, $i_{lim}$ is the limiting current density. 


### Calculating limiting current density

[The Engineering ToolBox](https://www.engineeringtoolbox.com/oxygen-solubility-water-d_841.html) provides oxygen solubility at different temperatures in sea water, which has a salinity of 35 ppt, or 35 g salt per kg sea water. At 25&deg; C and 1 bar, oxygen has a solubility of about 206 &mu;M. Since the partial pressure of oxygen in air is 20%, the concentration of oxygen in the bulk solution would be about 41.2 &mu;M, or 4.12E-5 M. Since there are $1000\ cm^3$ in a liter, $c_{\ce{O2},b} = 4.12E-5\ \frac{mol}{L} = 4.12E-5\ \frac{mol}{1000\ cm^3} = 4.12E-8\ \frac{mol}{cm^3}$. Nernst layer thickness, $\delta$, can range from 0.1 mm to 0.001 mm depending on stirring rate. The water in this case is not stirred, so I will start with $\delta = 0.1\ mm = 0.01\ cm$ as a preliminary estimate. The oxygen reduction reaction (ORR) has the following form

$$
\ce{O2 + 2H2O + 4e- -> 4OH-}
$$

so $z = 4$. Finally, the diffusion coefficient of oxygen in water at 25&deg; C, also from [The Engineering ToolBox](https://www.engineeringtoolbox.com/diffusion-coefficients-d_1404.html), is $D = 2.42E-5 \frac{cm^2}{s}$. These values can all then be plugged into the limiting current density equation:

$$
i_{lim} = -zFD_{\ce{O2}}\frac{c_{\ce{O2},b}}{\delta} = -4*96485*2.42E-5*\frac{4.12E-8}{0.01} = -3.85E-5 \frac{A}{cm^2}
$$


In [2]:
def i_lim_example():
    c_bulk = 4.12E-8
    delta = 0.01
    F = 96485
    z = 4
    D = 2.42E-5
    i_lim = -z * F * D * c_bulk / delta
    return i_lim

print(i_lim_example())

-3.847976176e-05


### Converting to moles of iron oxidized

Initially, I will assume that the reduction of oxygen is the dominating cathodic reaction, and will ignore the contribution of hydrogen evolution to the cathodic current. Since the net charge of the corroding system remains constant at zero, the anodic current must have equal magnitude to the cathodic current. This means that the current from iron oxidation must be equal to the current from oxygen reduction (with opposite sign). 

$$
|i_{red}| = i_{ox} = 3.85E-5
$$

I think the subscripts in $i_{\ce{red}}$ and $i_{\ce{ox}}$ could be a little confusing, since $ox$ refers to the oxidation of iron and not oxygen itself. I will instead use $i_{c}$ for the cathodic (reduction) current density and $i_{a}$ for the anodic (oxidation) current density. 

$$
|i_{c}| = i_{a} = 3.85E-5
$$

We can then use Faraday's equation to convert the current density back into an oxidation rate. First, we need the iron oxidation reaction:

$$
\ce{Fe -> Fe2+ + 2e-}
$$

The iron oxidation reaction produces two electrons, compared to the four electrons consumed in the oxygen reduction reaction. In other words, $z = 2$ for iron and $z = 4$ for oxygen. We can rearrange Faraday's law above and calculate the oxidation rate of iron per square centimeter using the anodic current density.

$$
i_{a} = \frac{zFn_{\ce{Fe}}}{At} \\
\frac{n_{\ce{Fe}}}{At} = \frac{i_{a}}{zF} = \frac{3.85E-5}{(2)(96485)} = 1.99E-10 \frac{mol}{cm^2s}
$$



In [3]:
def iron_ox_example():
    F = 96485
    z = 2
    i_lim = i_lim_example()
    return i_lim / (F * z)

print(iron_ox_example())

-1.99408e-10


## Important reactions for iron rust formation

The first important reaction, perhaps somewhat obviously, is the oxidation of iron metal by oxygen in the presence of water:

$$
\ce{2Fe + O2 + 2H2O -> 2Fe^{2+} + 4OH-}
$$

The two half-cell reactions for this redox reaction are:

$$
\ce{Fe -> Fe^{2} + 2e-}
\ce{O2 + 2H2O + 4e- -> 4OH-}
$$

Another important redox reaction is the further oxidation of $\ce{Fe^{2+}}$ in the presence of oxygen:

$$
\ce{4Fe^{2+} + O2 + 2H2O -> 4Fe^{3+} + 4OH-}
$$

The oxygen reduction half-cell reaction is the same, while the iron oxidation half-cell reaction is:

$$
\ce{Fe^{2+} -> Fe^{3+} + e-}
$$

The ions $\ce{Fe^{2+}}$ and $\ce{Fe^{3+}}$ can further react with $\ce{OH-}$ to form iron hydroxides:

$$
\ce{Fe^{2+} + 2OH- -> Fe(OH)2} \\
\ce{Fe^{3+} + 2OH- -> Fe(OH)3}
$$

Iron(III) hydroxide can dehydrate, even in the presence of water, to form iron oxyhydroxide:

$$
\ce{Fe(OH)3 -> FeOOH + H2O}
$$

Iron hydroxide is, from what I have read, one of the primary compounds forming the outer rust layer. It has the orange-brown color typically associated with rust. One more important compound formed during corrosion is magnetite, $\ce{Fe3O4}$, which has a complicated mechanism for formation, but some possible simplified reactions are the following:

$$
\ce{Fe(OH)2 + 2Fe(OH)3 -> Fe3O4 + 4H2O} \\
\ce{Fe^{2+} + 2Fe^{3+} + 8OH- -> Fe3O4 + 4H2O}
$$

Magnetite typically forms closer to the surface of the corroding metal and is denser than other rust products such as iron oxyhydroxide. In addition, it can also transfer charge, and studies have shown that the corrosion current density of iron with a layer of magnetite on top is equal to the limiting current density from oxygen diffusion through water, indicating that magnetite alone does not have a passivating effect.



## Rust layer composition

![title](rust_layer_composition.jpg)

## Corrosion product solubilities and reaction rates

The two iron hydroxides, $\ce{Fe(OH)2}$ and $\ce{Fe(OH)3}$ have limited solubility in water according to the following equilibriums:

$$
\ce{Fe(OH)2 <=> Fe^{2+} + 2OH-} \\ 
K_{\ce{Fe^{2+}}} = [\ce{Fe^{2+}}][\ce{OH-}]^2 = 10^{-14.5}\ \frac{mol^3}{dm^9} = 10^{-23.5}\ \frac{mol^3}{cm^9} \\
\ce{Fe(OH)3 <=> Fe^{3+} + 3OH-} \\
K_{\ce{Fe^{3+}}} = [\ce{Fe^{3+}}][\ce{OH-}]^3 = 10^{-38}\ \frac{mol^4}{dm^12} = 10^{-50}\ \frac{mol^3}{cm^9}
$$

The oxidation of iron(II) to iron(III) in neutral and alkaline solutions progresses according to the following rate equation:

$$ 
\frac{-d[\ce{Fe^{2+}}]}{dt} = k_0[\ce{Fe^{2+}}][\ce{O2}][\ce{OH-}]^2 \\
k_0 = 2.3E+14\ \frac{dm^9}{mol^3s} = 2.3E+23\ \frac{cm^9}{mol^3s}
$$


